# Capstone Project - Week 3

## Section 1: Scraping data from the wikipedia page:
Neighbourhood date from Toronto

In [1]:
# Libraries
import pandas as pd
import numpy as np

In [2]:
# getting the website data. Return is in html format.

import requests 

URL = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
WebCanada = requests.get(URL) 

# If you need to see the content
#print(WebCanada.content)

In [3]:
# In order to get data out of a html nested data, use parser. 

from bs4 import BeautifulSoup 
#from bs4 import comment
SoupCanada = BeautifulSoup(WebCanada.content, 'html5lib') # Other options for htmlflib =>  lxml, html.parser
#print(SoupCanada.prettify()) 

# Find the dive class id for the table. Then use that id to extract that part using .find 
# This will limit the data extraction to the particular table we are interested in.
TableCanada = SoupCanada.find('div', attrs = {'id':'mw-content-text'})
#TableCanada.prettify()

# Addtional: This is to show the page as it is.
# from IPython.core.display import display, HTML
# display(HTML(TableCanada.prettify()))

In [4]:
# Lists to store data from extraction
ListPostalCodes   = []
ListBoroughs      = []
ListNeighborhoods = []

In [5]:
# Get data from HTML file to lists
for row in TableCanada.find('table').find_all('tr'):
    cells = row.find_all('td')
    if(len(cells) > 0):                             # if there is a observation in a cell
        ListPostalCodes.append(cells[0].text.rstrip('\n'))
        ListBoroughs.append(cells[1].text.rstrip('\n'))
        ListNeighborhoods.append(cells[2].text.rstrip('\n'))
                # to avoid new lines in neighborhood cell. if there is few 

In [6]:
# create a the dataframe using the lists
TorontoDF = pd.DataFrame({ "PostalCode":   ListPostalCodes,
                           "Borough":      ListBoroughs,
                           "Neighborhood": ListNeighborhoods})

TorontoDF.head()


,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,
1,M2A,Not assigned,
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront


In [7]:
# Cleaning the Table

# Remove \n -------------------------------
TorontoDF['PostalCode'] = TorontoDF['PostalCode'].str.replace(r'\n', '')
TorontoDF['Borough'] = TorontoDF['Borough'].str.replace(r'\n', '')
TorontoDF.head()

# Remove not assigned -------------------
# Get names of indexes for which column Borough = not assigned
IndexNames = TorontoDF[ TorontoDF['Borough'] == 'Not assigned' ].index 
TorontoDF.drop(IndexNames , inplace=True)  # Delete these row indexes from dataFrame
TorontoDF.head()

# convert / into ,  -------------------
TorontoDF['Neighborhood'] = TorontoDF['Neighborhood'].str.replace(r' /', ',')
TorontoDF.head()

# Covert No assigned neighbourhood to borough name as stated in the assignment -------------------
TorontoDF.Neighborhood.replace('Not assigned',TorontoDF.Borough,inplace=True)
TorontoDF.head()

# Combine postcode -------------------
TorontoDF = TorontoDF.groupby(['PostalCode','Borough'], as_index=False).agg(lambda x: ','.join(x))
TorontoDF.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [8]:
#  Size of the dataframe
TorontoDF.shape

(103, 3)

## Section 2: Adding the geocode to the table

In [9]:
#Read the csv file and get to a dataframe
GeocodeDF = pd.read_csv('http://cocl.us/Geospatial_data')
GeocodeDF.rename(columns={'Postal Code': 'PostalCode'}, inplace=True)
GeocodeDF.head()

# Merge two data sets based on PostalCode
TorontoFinalDF = pd.merge(TorontoDF, GeocodeDF, on = 'PostalCode')
TorontoFinalDF.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


## Section 3: Creating a map of Toronto
In this section, I will use data from the wikipedia page, Foursqaure and the folium map to create a clustered map of Toronto neighbourhood

In [10]:
# Getting required libraries

import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

# uncomment this line if you haven't completed the Foursquare API lab
!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

# uncomment this line if you haven't completed the Foursquare API lab
!conda install -c conda-forge folium=0.5.0 --yes 
import folium # map rendering library

print('Libraries imported.')

Solving environment: ...working... done

# All requested packages already installed.

Solving environment: ...working... done

# All requested packages already installed.

Libraries imported.


In [11]:
#### Step 3.1 Creating and Cleaning the data Tabel

In [12]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(TorontoFinalDF['Borough'].unique()),
        TorontoFinalDF.shape[0]
    )
)


The dataframe has 10 boroughs and 103 neighborhoods.


In [13]:
# Keep only Toronto neighbourhood
TorontoBoroughs = TorontoFinalDF[TorontoFinalDF['Borough'].str.contains('Toronto')].reset_index(drop=True)
TorontoBoroughs.head()


,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
2,M4L,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.315572
3,M4M,East Toronto,Studio District,43.659526,-79.340923
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


#### Step 3.2 Obtaining Latitudes and Longitudes for Toronto City

In [14]:
# Use address to get lat and long
address = 'Toronto, Canada'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


#### Step 3.3 Creating a map of Toronto and superimposed borough data

In [15]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(TorontoBoroughs['Latitude'], TorontoBoroughs['Longitude'], TorontoBoroughs['Borough'], TorontoBoroughs['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

#### Step 3.4 Utilizing Foursqaure API 

In [16]:
CLIENT_ID = '' # your Foursquare ID
CLIENT_SECRET = '' # your Foursquare Secret
VERSION = '20180604' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: WASCKVHUH1MEKSL32EKDDMBRLKTXFTN5CDTNE20CFI504LKX
CLIENT_SECRET:51INCZPGU1FKYN2R2PVYVRE52O0HH15VHPPU3ZZXW0ZC3Q35


#### Step 3.4 Function to get neighbourhood venues and getting the data using TorontoBoroughs

In [17]:
def getNearbyVenues(names, latitudes, longitudes, postalcode, radius, LIMIT):
    
    venues_list=[] 
    
    for name, lat, lng, post in zip(names, latitudes, longitudes, postalcode):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            post,
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Postal Code',
                  'Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [18]:
# Running the above function for each neighbourhood

Toronto_Venues = getNearbyVenues(names=TorontoBoroughs['Neighborhood'],
                                   latitudes=TorontoBoroughs['Latitude'],
                                   longitudes=TorontoBoroughs['Longitude'],
                                   postalcode = TorontoBoroughs['PostalCode'],
                                   radius =500,
                                   LIMIT = 200
                                  )

The Beaches
The Danforth West, Riverdale
India Bazaar, The Beaches West
Studio District
Lawrence Park
Davisville North
North Toronto West
Davisville
Moore Park, Summerhill East
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
Rosedale
St. James Town, Cabbagetown
Church and Wellesley
Regent Park, Harbourfront
Garden District, Ryerson
St. James Town
Berczy Park
Central Bay Street
Richmond, Adelaide, King
Harbourfront East, Union Station, Toronto Islands
Toronto Dominion Centre, Design Exchange
Commerce Court, Victoria Hotel
Roselawn
Forest Hill North & West
The Annex, North Midtown, Yorkville
University of Toronto, Harbord
Kensington Market, Chinatown, Grange Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
Stn A PO Boxes
First Canadian Place, Underground city
Christie
Dufferin, Dovercourt Village
Little Portugal, Trinity
Brockton, Parkdale Village, Exhibition Place
High Park, The Junction South
Parkdale, Ron

In [19]:
print(Toronto_Venues.shape)
Toronto_Venues.head()

(1623, 8)


,Postal Code,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,M4E,The Beaches,43.676357,-79.293031,Glen Manor Ravine,43.676821,-79.293942,Trail
1,M4E,The Beaches,43.676357,-79.293031,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
2,M4E,The Beaches,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
3,M4E,The Beaches,43.676357,-79.293031,Glen Stewart Park,43.675278,-79.294647,Park
4,M4E,The Beaches,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood


In [20]:
# how many observation of venues for each neighbourhood (per unique Postal Code)
Toronto_Venues.groupby('Postal Code').count()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Postal Code,,,,,,,
M4E,5,5,5,5,5,5,5
M4K,43,43,43,43,43,43,43
M4L,23,23,23,23,23,23,23
M4M,40,40,40,40,40,40,40
M4N,3,3,3,3,3,3,3
M4P,8,8,8,8,8,8,8
M4R,22,22,22,22,22,22,22
M4S,35,35,35,35,35,35,35
M4T,1,1,1,1,1,1,1


In [21]:
print('There are {} uniques categories.'.format(len(Toronto_Venues['Venue Category'].unique())))

There are 226 uniques categories.


#### Step 3.5 Analyzing the neighbourhoods
##### NOTE: I add 'POSTALCODE' instead of 'neighborhood'becuase that where I have the unique Lattitude and Longitude

In [22]:
# one hot encoding -  Get dummies based on categories
Toronto_onehot = pd.get_dummies(Toronto_Venues[['Venue Category']], prefix="", prefix_sep="")

# add postalcode column back to dataframe. 
Toronto_onehot['PostalCode'] = Toronto_Venues['Postal Code'] 

# move postalcode column to the first column - CHECK
fixed_columns = [Toronto_onehot.columns[-1]] + list(Toronto_onehot.columns[:-1])
Toronto_onehot = Toronto_onehot[fixed_columns]


Toronto_onehot.shape
Toronto_onehot.head()


,PostalCode,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Butcher,Cable Car,Café,Cajun / Creole Restaurant,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Convention Center,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hospital,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Poke Place,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Snack Place,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Swim School,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Women's Store,Yoga Studio
0,M4E,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
1,M4E,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [23]:
# Group rows by neighborhood and take the mean of frequency for each category
Toronto_grouped = Toronto_onehot.groupby('PostalCode').mean().reset_index()
Toronto_grouped.shape


(39, 227)

In [24]:
# Getting top 5 venues of each postalcode

num_top_venues = 5

for hood in Toronto_grouped['PostalCode']:
    print("----"+hood+"----")
    temp = Toronto_grouped[Toronto_grouped['PostalCode'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----M4E----
               venue  freq
0               Park   0.2
1  Health Food Store   0.2
2                Pub   0.2
3       Neighborhood   0.2
4              Trail   0.2


----M4K----
                    venue  freq
0        Greek Restaurant  0.19
1      Italian Restaurant  0.07
2             Coffee Shop  0.07
3  Furniture / Home Store  0.05
4          Ice Cream Shop  0.05


----M4L----
                  venue  freq
0        Sandwich Place  0.09
1           Pizza Place  0.09
2  Fast Food Restaurant  0.09
3         Burrito Place  0.04
4            Steakhouse  0.04


----M4M----
                 venue  freq
0                 Café  0.10
1          Coffee Shop  0.08
2              Brewery  0.05
3               Bakery  0.05
4  American Restaurant  0.05


----M4N----
                             venue  freq
0                             Park  0.33
1                         Bus Line  0.33
2                      Swim School  0.33
3  Molecular Gastronomy Restaurant  0.00
4                  

In [25]:
# function to sort venues in decending order
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [26]:
num_top_venues = 5

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['PostalCode']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['PostalCode'] = Toronto_grouped['PostalCode']

for ind in np.arange(Toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,PostalCode,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,M4E,Pub,Trail,Health Food Store,Neighborhood,Park
1,M4K,Greek Restaurant,Italian Restaurant,Coffee Shop,Furniture / Home Store,Bookstore
2,M4L,Fast Food Restaurant,Sandwich Place,Pizza Place,Brewery,Ice Cream Shop
3,M4M,Café,Coffee Shop,Gastropub,Brewery,Bakery
4,M4N,Park,Bus Line,Swim School,Yoga Studio,Deli / Bodega


#### Step 3.6 Clustering

In [27]:
# set number of clusters
kclusters = 3

Toronto_grouped_clustering = Toronto_grouped.drop('PostalCode', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([2, 0, 0, 0, 2, 0, 0, 0, 2, 0])

In [28]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

Toronto_merged = TorontoBoroughs

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
Toronto_merged = Toronto_merged.join(neighborhoods_venues_sorted.set_index('PostalCode'), on='PostalCode')

Toronto_merged.head() # check the last columns!

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,M4E,East Toronto,The Beaches,43.676357,-79.293031,2,Pub,Trail,Health Food Store,Neighborhood,Park
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188,0,Greek Restaurant,Italian Restaurant,Coffee Shop,Furniture / Home Store,Bookstore
2,M4L,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.315572,0,Fast Food Restaurant,Sandwich Place,Pizza Place,Brewery,Ice Cream Shop
3,M4M,East Toronto,Studio District,43.659526,-79.340923,0,Café,Coffee Shop,Gastropub,Brewery,Bakery
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,2,Park,Bus Line,Swim School,Yoga Studio,Deli / Bodega


In [29]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Toronto_merged['Latitude'], Toronto_merged['Longitude'], Toronto_merged['PostalCode'], Toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

#### Step 3.7: Understanding the Clusters

##### Cluster 1 -------------------------

In [30]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 0, Toronto_merged.columns[[1] + list(range(3, Toronto_merged.shape[1]))]]

,Borough,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
1,East Toronto,43.679557,-79.352188,0,Greek Restaurant,Italian Restaurant,Coffee Shop,Furniture / Home Store,Bookstore
2,East Toronto,43.668999,-79.315572,0,Fast Food Restaurant,Sandwich Place,Pizza Place,Brewery,Ice Cream Shop
3,East Toronto,43.659526,-79.340923,0,Café,Coffee Shop,Gastropub,Brewery,Bakery
5,Central Toronto,43.712751,-79.390197,0,Dance Studio,Gym,Department Store,Food & Drink Shop,Sandwich Place
6,Central Toronto,43.715383,-79.405678,0,Clothing Store,Coffee Shop,Yoga Studio,Restaurant,Rental Car Location
7,Central Toronto,43.704324,-79.388790,0,Dessert Shop,Sandwich Place,Pizza Place,Italian Restaurant,Café
9,Central Toronto,43.686412,-79.400049,0,Coffee Shop,Pub,Supermarket,Bank,Sports Bar
11,Downtown Toronto,43.667967,-79.367675,0,Restaurant,Coffee Shop,Café,Italian Restaurant,Bakery
12,Downtown Toronto,43.665860,-79.383160,0,Sushi Restaurant,Japanese Restaurant,Coffee Shop,Gay Bar,Restaurant
13,Downtown Toronto,43.654260,-79.360636,0,Coffee Shop,Pub,Bakery,Park,Café


##### Cluster 2 ----------------------

In [31]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 1, Toronto_merged.columns[[1] + list(range(3, Toronto_merged.shape[1]))]]

,Borough,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
22,Central Toronto,43.711695,-79.416936,1,Garden,Yoga Studio,Deli / Bodega,Electronics Store,Eastern European Restaurant


##### Cluster 3 ---------------------

In [32]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 2, Toronto_merged.columns[[1] + list(range(3, Toronto_merged.shape[1]))]]

,Borough,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,East Toronto,43.676357,-79.293031,2,Pub,Trail,Health Food Store,Neighborhood,Park
4,Central Toronto,43.728020,-79.388790,2,Park,Bus Line,Swim School,Yoga Studio,Deli / Bodega
8,Central Toronto,43.689574,-79.383160,2,Playground,Yoga Studio,Dance Studio,Eastern European Restaurant,Dumpling Restaurant
10,Downtown Toronto,43.679563,-79.377529,2,Park,Trail,Playground,Yoga Studio,Cupcake Shop
23,Central Toronto,43.696948,-79.411307,2,Jewelry Store,Trail,Mexican Restaurant,Sushi Restaurant,Department Store


#### END